In [18]:
#Sample code

import cvxpy as cp
import numpy as np

m=30
n=20
np.random.seed(1)

#create ax<=b?
A= np.random.randn(m,n)
b=np.random.randn(m)

x=cp.Variable(n)
objective= cp.Minimize(cp.sum_squares(A*x-b))
constraints= [0<=x, x<=1]
prob= cp.Problem(objective, constraints)

result=prob.solve()
print(x.value)


[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]


C:\Users\Windows\Anaconda3\envs\cvxpy\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Windows\Anaconda3\envs\cvxpy\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [ ]:
# Code Structure

#Data Class
    #Pulls data that we need (arguments: stock tickers, start date, end_date, frequency; outputs pandas DF w/ closing prices)

#Parameter Estimation Class

    ##Averaging Methods (arguments: time, x- day history; retrieves historic data from Data Class, uses averaging and covariance)

    ##Factor Model Methods:
        #Regression function, pass in different factors to the same function for Fama French and other factor models
        
        #Call the Averaging Methods on the factors (expectation, covariance factors)
        
        #Return and Covariance Function (takes in the factor loadings and factor statistics;returns asset returns and covariance)
        #Question at time t, how we do we estimate returns at t+1, t+2... doesn't make sense to Monte Carlo 
        
        #(assumes static returns with some noise, but we want EXPECTED returns not returns+noise).. Perhaps some predictive modelling?
        #assume static covariance of assets in multiperiod model.. is this a reasonable assumption?

#Optimization Model Class
    #Takes in risk model, constraint, cost model object, returns optimal set of weights

#Risk Models Class (base)

    ##Variance xTQx (Implement first as baseline, inherits from Risk Model Class)
    
    ##Robustness (Inherits from Variance? Just adding delta)
    
    ##CVaR (need to use another optimization model for this).. At each time t, simulate 5000 return paths of length H per asset
    
#Constraint Class
    ##Constaints static methods that just returns the appropriate constraint

#Transaction Models Class:
    ##For now just a simple method that takes in paramaters bid ask spread, asset volatlity, volume traded, returns a cost
    
#Holding Models Class
    ##For now just a simple method that takes fees for short position, returns holdings

#Benchmark Class:
    ##Variety of Functions that returns a set of weights at each time step
    
    ##Stock Indices -- How would I find the weights of S&P over time for example, can just pull from a site?
    
    #Risk Parity -- Find portfolio with equal risk attribution
    
    ##Problem with stock indices, risk parity, if i want to optimize with active risk, do I need to predict
    
    ##Risk free, weight of cash component is always 1
    
    #equal weight.. static distribution of weights    
    

#Performance Metrics Class
    ##Average Return Method (pass in array of returns, return average)
    ##Realized Volatility Method (pass in array of returns, find volatility)
    ##Information Ratio Method (pass in Returns, Volatiliy, returns IR)
    

#Backtesting Class
    ##Simulate method (give start date, end date, rebalance frequency, lookahead period H):
        ###Loop from start date to end date, step size rebalance frequency
            ####At each step use solver, 


In [2]:
#Imported Libraries
import cvxpy as cp
import numpy as np
from sklearn import datasets
from scipy import random, linalg

In [3]:
#now create a simple single period MVO

#Assume universe 3 stocks
num_stocks=3

#Need 3x1 vector r of expected returns, need covariance matrix Q (randomly initiate for now)
np.random.seed(1)
r= np.random.randn(num_stocks)/10
Q= datasets.make_spd_matrix(num_stocks)


#Construct optimization problem
x=cp.Variable(num_stocks)
lam=0.1
portfolio_return= r.transpose()@x
portfolio_risk = cp.quad_form(x,Q)
objective= cp.Maximize(portfolio_return - lam*portfolio_risk)

#Set Constraints
constraints=[cp.sum(x)==1]

#Construct Problem and Solve
prob= cp.Problem(objective, constraints)
result=prob.solve()


In [4]:
print(result)
print(x.value)

0.48634677108640123
[ 4.6597536  -2.21625584 -1.44349776]


In [10]:
#create a simple multi period MVO, 5 step lookahead
look_ahead=5

#Assume universe 3 stocks
num_stocks=3

#Need 3x5 vector r of expected returns, need 5 covariance matrix Q (randomly initiate for now)
np.random.seed(1)
r= np.random.randn(num_stocks,look_ahead)/10
Q= [datasets.make_spd_matrix(num_stocks) for _ in range(look_ahead)]


#Construct optimization problem
x=cp.Variable(num_stocks,look_ahead)
lam=0.1
portfolio_return_per_period= r.transpose()@x
portfolio_return= cp.sum(portfolio_return_per_period)

portfolio_risk=0
for i in range(look_ahead):
    portfolio_risk+=cp.quad_form(x[:,i],Q[i])
    

objective= cp.Maximize(portfolio_return - lam*portfolio_risk)

#Set Constraints
constraints=[cp.sum(x)==1]

#Construct Problem and Solve
prob= cp.Problem(objective, constraints)
result=prob.solve()




TypeError: Variable name 5 must be a string.

In [24]:
look_ahead=5
num_stocks=3
x=cp.Variable((num_stocks,look_ahead))
test=cp.quad_form(x[:,0],Q[0])

In [39]:
#now create a sample single period MVO
#Assume universe 3 stocks
num_stocks=3

#Need 10x1 vector r of expected returns, need covariance matrix Q (randomly initiate for now)

Q=[[0.8, 0.05, -0.1],[0.05, 0.6, 0.08], [-0.1 ,0.08 ,0.7]]
#Construct optimization problem
x=cp.Variable(num_stocks)
lam=0.1
portfolio_return= r.transpose()@x
portfolio_risk = cp.quad_form(x,Q)
objective= cp.Minimize(portfolio_risk)

#Set Constraints
constraints=[cp.sum(x)==1]

#Construct Problem and Solve
prob= cp.Problem(objective, constraints)
prob.solve()

0.23963274106014432

In [42]:
x.value

array([0.32309865, 0.32562635, 0.351275  ])

In [59]:
a=[[1,2],[3,4]]
b=[[5,6],[7,8]]

res=np.stack((a,b))
res[:][0][:]

array([[1, 5],
       [2, 6]])